In [1]:
import pandas as pd
import cpca
from datetime import datetime

In [2]:
read_csv_path = './hunter-info.csv'
save_csv_path = './result1-2.csv'

frame_data = pd.read_csv(read_csv_path, encoding='GBK')

In [3]:
frame_data.head(3)

,hunter_id,hunter_name,hunter_sex,hunter_bthday,hunter_addr,hunter_exp,hunter_sociality,hunter_eval,exp_position,exp_min_wage,...,exp_report,resume_keys,job_exps,project_exps,competition_exps,education_exps,training_exps,skill_exps,language_exps,cert_exps
0,1632947816114552832,梁先生,0.0,2001-05-31T16:00:00.000Z,广东省东莞市东莞,1年工作经验,LEAGUE_MEMBER,熟练操作python、爬虫、sql语句、excel和powerbi等软件,"['数据分析师', '数据挖掘工程师', '机器学习工程师']",6000,...,随时到岗,[],['广东泰迪智能科技有限公司[python数据分析实习生]:1.\t数据库分析与数据储存：数...,['天猫用户流失率分析[成员]:对天猫用户的用户行为：浏览、购买、收藏、加购物车从时间维度上...,"['泰迪杯[三等奖]:第四届泰迪杯数据分析技能赛', '其他[南方赛区一等奖]:全国财经大数...",['广东第二师范学院[本科]:统计学'],[],"['python编程软件[SKILLED]', 'excel[SKILLED]', 'pow...",['英语[GOOD]'],"['驾驶证[其他]', 'CET4[其他]', 'Python技术应用工程师（高级）[工信部..."
1,1632286586903330816,钟先生,0.0,1998-02-28T16:00:00.000Z,广东省广州市天河区,无经验,LEAGUE_MEMBER,"本人是一名退伍军人,保留军人本色,为人平和,对待工作严谨,具备敏锐的观察力,对数字敏感,熟悉...","['数据分析师', '数据挖掘工程师', '机器学习工程师']",4000,...,随时到岗,[],[],[],[],[],[],[],[],[]
2,1579833809761861632,李先生,0.0,2000-05-29 00:00:00.0,广东省中山市中山,无经验,MASSES,担任过班干、技术部部长、项目负责人因此具有一定组织力，技术方面：荣获python编程国赛三等...,"['数据分析师', '数据挖掘工程师']",4000,...,时间待议,"['python', '数据分析', '机器学习', '深度学习']",['广东泰迪智能科技股份有限公司[助教]:人工智能方面的技术支持，包括软件安装、理论知识和课...,['银行客户忠诚度分析[数据分析]:1对客户数据进行预处理，并对字符型数据进行特征编码。2基...,"['泰迪杯[一等奖]:第四届泰迪杯数据分析技能赛', '其他[铜牌]:“第七届中国国际“互联...",[],[],"['python[MASTER]', '数据分析[SKILLED]', '数据建模（机器学习...",[],['1+X中级数据分析师[其他]']


In [4]:
for hunter_sex in frame_data['hunter_sex'].unique():
    if pd.isna(hunter_sex): continue
    print(hunter_sex, frame_data[frame_data['hunter_sex'] == hunter_sex].iloc[0, 0])

hunter_sex_kind_json = {0: '男', 1: '女'}

0.0 1632947816114552832
1.0 1630884821167374336


In [5]:
for exp_require_kind in frame_data['exp_require_kind'].unique():
    if pd.isna(exp_require_kind): continue
    print(exp_require_kind, frame_data[frame_data['exp_require_kind'] == exp_require_kind].iloc[0, 0])

exp_require_json = {0: '实习', 1: '无', 2: '全职'}

2.0 1632947816114552832
0.0 1632286586903330816
1.0 1484409030448381952


In [6]:
for hunter_sociality in frame_data['hunter_sociality'].unique():
    if pd.isna(hunter_sociality): continue
    print(hunter_sociality, frame_data[frame_data['hunter_sociality'] == hunter_sociality].iloc[0, 0])

require_kind_json = { 'MASSES': '群众', 'LEAGUE_MEMBER': '共青团员', 'PARTY_MEMBER': '党员', 'PROBATIONARY_PARTY_MEMBER': '预备党员'}

LEAGUE_MEMBER 1632947816114552832
MASSES 1579833809761861632
PARTY_MEMBER 1627308899029876736
PROBATIONARY_PARTY_MEMBER 1583024721031725056


In [7]:
save_frame = {"序号": [], '求职者 ID': [], '姓名': [], '性别': [], '年龄': [], '出生地': [], '社会属性': [], '工作经验': [], 
              '期望岗位': [], '期望薪资': [], '期望类型': [], '期望城市': [], '期望行业': [], '预计到岗': [], '工作经历': [], 
              '项目经历': [], '竞赛经历': [], '教育经历': [], '培训经历': [], '技能类型': [], '语言类型': [], '证书类型': []}

TEST, RECORD, count = True, 5, 0
size = frame_data.shape[0]

for inx in range(size):
    if TEST and inx % RECORD == 0: continue
    item = frame_data.iloc[inx, :]
    count += 1
    save_frame['序号'].append(count)
    save_frame['求职者 ID'].append('\t' + str(item['hunter_id']))
    save_frame['姓名'].append(item['hunter_name'])
    save_frame['性别'].append(hunter_sex_kind_json[item['hunter_sex']] if not pd.isna(item['hunter_sex']) else '')
    try:
        age = item['hunter_bthday'].replace('-', '/').replace('T', ' ').split(' ')[0]
        begin_year = datetime.strptime(age, '%Y/%m/%d').year
        end_year = datetime.now().year
        age = end_year - begin_year
    except:
        age = ''
    save_frame['年龄'].append(age)
    try:
        addr = cpca.transform([item['hunter_addr']])
        addr = ''.join([addr['省'][0], addr['市'][0]]).replace('市县', '市')
    except:
        addr = ''
    save_frame['出生地'].append(addr)
    save_frame['社会属性'].append(require_kind_json[item['hunter_sociality']] if not pd.isna(item['hunter_sociality']) else '')
    try:
        work_year = '无' if '无经验' in item['hunter_exp'] else (
            item['hunter_exp'].replace('工作经验', '') if '工作经验' in item['hunter_exp'] else item['hunter_exp'])
    except:
        work_year = ''
    save_frame['工作经验'].append(work_year)
    save_frame['期望岗位'].append(item['exp_position'])
    save_frame['期望薪资'].append([item['exp_min_wage'], item['exp_max_wage']])
    save_frame['期望类型'].append(exp_require_json[item['exp_require_kind']] if not pd.isna(item['exp_require_kind']) else '')
    save_frame['期望城市'].append(item['exp_city'])
    save_frame['期望行业'].append(item['exp_industry'])
    save_frame['预计到岗'].append(item['exp_report'].replace('到岗', '') if not pd.isna(item['exp_report']) else '')
    save_frame['工作经历'].append([exp.split(':')[0] for exp in eval(item['job_exps'])])
    save_frame['项目经历'].append([exp.split(':')[0] for exp in eval(item['project_exps'])])
    save_frame['竞赛经历'].append([exp.split(':')[0] for exp in eval(item['competition_exps'])])
    save_frame['教育经历'].append([exp.split(':')[0] for exp in eval(item['education_exps'])])
    save_frame['培训经历'].append(eval(item['training_exps']))
    save_frame['技能类型'].append(eval(item['skill_exps']))
    save_frame['语言类型'].append(eval(item['language_exps']))
    save_frame['证书类型'].append(eval(item['cert_exps']))

print(size, len(save_frame['序号']))
# for key, value in save_frame.items():
#     print(key, len(value))

save_frame = pd.DataFrame(save_frame)
save_frame.to_csv(save_csv_path, index=False, encoding="GBK", errors='ignore')

10863 8690
